<table>
    <tr>
        <td><img src="./imagenes/Macc.png" width="400"/></td>
        <td>&nbsp;</td>
        <td>
            <h1 style="color:blue;text-align:left">Inteligencia Artificial</h1></td>
        <td>
            <table><tr>
            <tp><p style="font-size:150%;text-align:center">Notebook 7</p></tp>
            <tp><p style="font-size:150%;text-align:center">Lógica Proposicional </p></tp>
            </tr></table>
        </td>
    </tr>
</table>

---


## Objetivo <a class="anchor" id="inicio"></a>

En este notebook veremos una manera de implementar una base de conocimiento sobre la cual se ejecutan los algoritmos de deducción forward-chaining y backward-chaining. 

## Secciones

Desarrollaremos la explicación en las siguientes secciones:

* [Base de conocimiento.](#base)
* [Forward chaining.](#forward)
* [Backward chaining.](#backward)


## Base de conocimiento <a class="anchor" id="base"></a>

([Volver al inicio](#inicio))

Una base de conocimiento es una estructura sobre la cual se pueden hacer consultas de objetivos. Más adelante usaremos las estrategias de forward y backward chaining para hacer las consultas. Lo que necesitamos ahora es una implementación de la base de conocimiento en python. 

Una base de conocimiento tiene dos partes: 

* Los datos, que son literales que ya sabemos como verdaderos; 
* Las reglas, que representan el conocimiento que se aplicará sobre los datos para obtener nuevos datos.

Implementaremos una base de conocimiento mediante la clase `LPQuery`, que se encuentra en la librería `logica`:

In [1]:
from logica import LPQuery

Inicializamos una base de conocimiento con el ejemplo de las diapositivas:

In [2]:
formulas = ['q>r','pYs>r','s','p']
base_ejemplo = LPQuery(formulas)
print(base_ejemplo)

Datos:
s
p

Reglas:
q>r
pYs>r



La clase `LPQuery` tiene los siguientes atributos:

* datos: lista de literales que ya sabemos como verdaderos.
* reglas: lista de reglas que representan el conocimiento.
* atomos: lista de los átomos que aparecen en los datos y las reglas.

También tiene los siguientes métodos:

* reglas_aplicables(`literal`): que devuelve una lista de reglas en la base cuya cabeza es el `literal`.
* test_objetivo(`literal`): que devuelve verdadero si el `literal` está en los datos. 
* TELL(`formula`): que incluye en la base de conocimiento la `formula`en el lugar adecuado, dependiendo de si es un dato o una regla.

In [3]:
reglas = base_ejemplo.reglas
print(reglas[1].nombre)
print(reglas[1].cuerpo)
print(reglas[1].cabeza)

pYs>r
['p', 's']
r


In [4]:
base_ejemplo.atomos

['q', 'r', 'p', 's']

## Forward chaining <a class="anchor" id="forward"></a>

([Volver al inicio](#inicio))

El siguiente es el pseudo código de la función `pl_fc_entails`, que implementa la estratégia de deducción forward chaining:

<img src="./imagenes/forward-chaining.png" width="550"/>

**Ejercicio 2:**

Implemente un código python con el algoritmo `pl_fc_entails`. Verifique cuáles de las letras 'p', 'q', 'r', o 's' se puede deducir a partir de la base de conocimiento creada anteriormente.

**Nota:** Observe que al inicializar `queue`como la lista de literales en los datos, es importante que `queue` sea una copia. Si se usa la instrucción `queue=base.datos`, entonces al modificar `queue` se modifica también `base.datos`. Para que no suceda esto, se puede utilizar `queue=deepcopy(base.datos)` (observe que hay que importar `deepcopy` de la libería `copy`).

La respuesta debe ser:

```
El átomo p se deduce de la base?: True
El átomo q se deduce de la base?: False
El átomo r se deduce de la base?: True
El átomo s se deduce de la base?: True
```

**Respuesta:**

Una posible implementación es la siguiente:

In [5]:
from copy import deepcopy

def pl_fc_entails(base, q) :
    count = {}
    for regla in base.reglas:
        count[regla.nombre] = len(regla.cuerpo)
    inferred = dict(zip(base.atomos, [False]*len(base.atomos)))
    queue = deepcopy(base.datos)
    while len(queue) > 0:
        p = queue.pop()
        if p == q:
            return True
        elif inferred[p] == False:
            inferred[p] = True
            for regla in base.reglas:
                if p in regla.cuerpo:
                    count[regla.nombre] -= 1
                    if count[regla.nombre] == 0:
                        queue.append(regla.cabeza)
    return False

In [6]:
print("El átomo p se deduce de la base?:", pl_fc_entails(base_ejemplo, 'p'))
print("El átomo q se deduce de la base?:", pl_fc_entails(base_ejemplo, 'q'))
print("El átomo r se deduce de la base?:", pl_fc_entails(base_ejemplo, 'r'))
print("El átomo s se deduce de la base?:", pl_fc_entails(base_ejemplo, 's'))

El átomo p se deduce de la base?: True
El átomo q se deduce de la base?: False
El átomo r se deduce de la base?: True
El átomo s se deduce de la base?: True


---

## Backward chaining <a class="anchor" id="backward"></a>

([Volver al inicio](#inicio))

El siguiente es el pseudocódigo de la función `and_or_graph_search`, el cual implementa la estratégia de deducción backward chaining:

<img src="./imagenes/and-or-graph-search.png" width="520"/>
<img src="./imagenes/or-search1.png" width="490"/>
<img src="./imagenes/and-search.png" width="500"/>


**Ejercicio 3:**

Implemente un código python con el algoritmo `and_or_graph_search`. Verifique cuáles de las letras 'p', 'q', 'r', o 's' se puede deducir a partir de la base de conocimiento creada anteriormente.

La respuesta debe ser:

```
El átomo p se deduce de la base?: success
El átomo q se deduce de la base?: failure
El átomo r se deduce de la base?: success
El átomo s se deduce de la base?: success
```

**Respuesta:**

Una posible implementación es la siguiente:

In [7]:
def and_or_graph_search(objetivo, base):
    return or_search(objetivo, base, [])

def or_search(head, base, camino):
    if base.test_objetivo(head):
        return 'success'
    elif head in camino:
        return 'failure'
    reglas = base.reglas_aplicables(head)
    if not reglas:
        return 'failure'
    for regla in reglas:
        plan = and_search(regla.cuerpo, base, [head] + camino)
        if plan != 'failure':
            return 'success'    
    return 'failure'

def and_search(literales, base, camino):
    for literal in literales:
        plan = or_search(literal, base, camino)
        if plan == 'failure':
            return 'failure'
    return 'success'

In [8]:
print("El átomo p se deduce de la base?:", and_or_graph_search('p', base_ejemplo))
print("El átomo q se deduce de la base?:", and_or_graph_search('q', base_ejemplo))
print("El átomo r se deduce de la base?:", and_or_graph_search('r', base_ejemplo))
print("El átomo s se deduce de la base?:", and_or_graph_search('s', base_ejemplo))

El átomo p se deduce de la base?: success
El átomo q se deduce de la base?: failure
El átomo r se deduce de la base?: success
El átomo s se deduce de la base?: success


---

## En este notebook usted aprendió

* Las reglas como un fragmento de la lógica proposicional para la deducción automática.
* El algoritmo de deducción mediante forward-chaining.
* El algoritmo de deducción mediante backward-chaining.
* La estructura de las bases de conocimiento.
